In [16]:
import pandas as pd
import numpy as np
import ast
import os

# Step 1: Read and parse the CSV file
df = pd.read_csv("/home/aleksis/PycharmProjects/DatascienceGroupProject/TheaterDataAnalysis/scripts/data/raw/theater_data4.csv")

# Extract 'id', 'size', and 'value'
ids = ast.literal_eval(df.loc[0, 'id'])
size = ast.literal_eval(df.loc[0, 'size'])
values = ast.literal_eval(df.loc[0, 'value'])

# Convert 'value' into an array and reshape
values_array = np.array(values, dtype=object)
values_array = values_array.reshape(size)

# Function to get labels for a dimension
def get_dimension_labels(df, dimension_name, size):
    index_to_label = {}
    for col in df.columns:
        if col.startswith(f'dimension.{dimension_name}.category.index.'):
            idx = col.split('.')[-1]
            index = int(df.loc[0, col])
            label_col = f'dimension.{dimension_name}.category.label.{idx}'
            label = df.loc[0, label_col]
            index_to_label[index] = label
    labels = [index_to_label[i] for i in range(size)]
    return labels

# Get labels for each dimension
year_labels = get_dimension_labels(df, 'Aasta', size[0])
sihtgrupp_labels = get_dimension_labels(df, 'Sihtgrupp', size[1])
naitaja_labels = get_dimension_labels(df, 'Näitaja', size[2])

# Create MultiIndex and DataFrame
index = pd.MultiIndex.from_product([year_labels, sihtgrupp_labels, naitaja_labels], names=ids)
data_flat = values_array.flatten()
df_final = pd.DataFrame(data=data_flat, index=index, columns=['value']).reset_index()

# Handle missing values
df_final['value'] = df_final['value'].replace({None: np.nan})
df_final['value'] = pd.to_numeric(df_final['value'], errors='coerce')

# Step 2: Pivot the DataFrame
df_pivot = df_final.pivot_table(
    index=['Aasta', 'Sihtgrupp'],
    columns='Näitaja',
    values='value',
    aggfunc='first'
).reset_index()

# Flatten the column MultiIndex
df_pivot.columns.name = None

# Step 3: Rename and rearrange columns
df_pivot.rename(columns={
    'Aasta': 'Year',
    'Sihtgrupp': 'Category',
    'Lavastused': 'Lavastused',
    '..uuslavastused': '..uuslavastused',
    'Etendused': 'Etendused',
    '..külalisetendused': '..külalisetendused',
    'Vaatajad, tuhat': 'Vaatajad, tuhat'
}, inplace=True)

df_pivot = df_pivot[['Year', 'Category', 'Lavastused', '..uuslavastused', 'Etendused', '..külalisetendused', 'Vaatajad, tuhat']]

# Step 4: Handle missing values and data types
df_pivot.fillna('..', inplace=True)

# Format numeric columns
numeric_columns = ['Lavastused', '..uuslavastused', 'Etendused', '..külalisetendused', 'Vaatajad, tuhat']

for col in numeric_columns:
    df_pivot[col] = df_pivot[col].apply(lambda x: '{:,}'.format(x).replace(',', ' ') if isinstance(x, (int, float)) else x)

# Step 5: Replace decimal dots with commas
df_pivot['Vaatajad, tuhat'] = df_pivot['Vaatajad, tuhat'].apply(lambda x: str(x).replace('.', ',') if isinstance(x, str) else x)

# Step 6: Adjust Year display
df_pivot.sort_values(by=['Year', 'Category'], inplace=True)
df_pivot.reset_index(drop=True, inplace=True)
df_pivot['Year'] = df_pivot['Year'].mask(df_pivot['Year'].duplicated())
df_pivot['Year'].fillna('', inplace=True)

# Step 7: Display the final DataFrame
print(df_pivot.to_string(index=False))


# Step 1: Ensure the directory exists
output_dir = '/home/aleksis/PycharmProjects/DatascienceGroupProject/TheaterDataAnalysis/data/processed'
os.makedirs(output_dir, exist_ok=True)

# Step 2: Define the output file path
output_csv_path = os.path.join(output_dir, 'cleaned_theater_data4.csv')
output_excel_path = os.path.join(output_dir, 'cleaned_theater_data4.xlsx')

# Step 3: Save as CSV with European formatting
df_pivot.to_csv(
    output_csv_path,
    index=False,
    encoding='utf-8',
    sep=';',        # Semicolon separator
    decimal=',',    # Comma decimal
    float_format='%.1f'  # One decimal place
)

# Step 4: Save as Excel file
df_pivot.to_excel(output_excel_path, index=False)

# Step 1: Read the CSV file
df = pd.read_csv("/home/aleksis/PycharmProjects/DatascienceGroupProject/TheaterDataAnalysis/scripts/data/raw/theater_data.csv")

# Step 2: Extract 'id', 'size', and 'value'
ids = ast.literal_eval(df.loc[0, 'id'])
size = ast.literal_eval(df.loc[0, 'size'])
values = ast.literal_eval(df.loc[0, 'value'])

# Step 3: Convert 'value' into an array and reshape
values_array = np.array(values, dtype=object)
values_array = values_array.reshape(size)

# Function to get labels for a dimension
def get_dimension_labels(df, dimension_name, size):
    index_to_label = {}
    for col in df.columns:
        if col.startswith(f'dimension.{dimension_name}.category.index.'):
            idx = col.split('.')[-1]
            index = int(df.loc[0, col])
            label_col = f'dimension.{dimension_name}.category.label.{idx}'
            label = df.loc[0, label_col]
            index_to_label[index] = label
    labels = [index_to_label[i] for i in range(size)]
    return labels

# Step 4: Get labels for each dimension
year_labels = get_dimension_labels(df, 'Aasta', size[0])
category_labels = get_dimension_labels(df, 'Kategooria', size[1])
indicator_labels = get_dimension_labels(df, 'Näitaja', size[2])

# Step 5: Create MultiIndex and DataFrame
index = pd.MultiIndex.from_product([year_labels, category_labels, indicator_labels], names=ids)
data_flat = values_array.flatten()
df_final = pd.DataFrame(data=data_flat, index=index, columns=['value']).reset_index()

# Step 6: Handle missing values
df_final['value'] = df_final['value'].replace({None: np.nan, 'None': np.nan})
df_final['value'] = pd.to_numeric(df_final['value'], errors='coerce')

# Step 7: Pivot the DataFrame
df_pivot = df_final.pivot_table(
    index=['Aasta', 'Kategooria'],
    columns='Näitaja',
    values='value',
    aggfunc='first'
).reset_index()

# Flatten the column MultiIndex
df_pivot.columns.name = None

# Step 8: Rename and reorder columns
df_pivot.rename(columns={
    'Aasta': 'Year',
    'Kategooria': 'Category',
    'Teatrite arv': 'Teatrite arv',
    'Lavastused': 'Lavastused',
    '..uuslavastused': '..uuslavastused',
    'Kultuuriüritused': 'Kultuuriüritused',
    'Etendused': 'Etendused',
    'Vaatajad, tuhat': 'Vaatajad, tuhat',
    'Teatriskäigud 1000 elaniku kohta': 'Teatriskäigud 1000 elaniku kohta',
    'Saalid': 'Saalid',
    'Istekohad': 'Istekohad'
}, inplace=True)

desired_columns = [
    'Year', 'Category', 'Teatrite arv', 'Lavastused', '..uuslavastused',
    'Kultuuriüritused', 'Etendused', 'Vaatajad, tuhat',
    'Teatriskäigud 1000 elaniku kohta', 'Saalid', 'Istekohad'
]
df_pivot = df_pivot[desired_columns]

# Step 9: Handle missing values and format data
df_pivot.fillna('..', inplace=True)

numeric_columns = [
    'Teatrite arv', 'Lavastused', '..uuslavastused',
    'Kultuuriüritused', 'Etendused', 'Vaatajad, tuhat',
    'Teatriskäigud 1000 elaniku kohta', 'Saalid', 'Istekohad'
]

for col in numeric_columns:
    df_pivot[col] = df_pivot[col].apply(lambda x: '{:,}'.format(x).replace(',', ' ') if isinstance(x, (int, float)) else x)

# Replace decimal dots with commas
df_pivot['Vaatajad, tuhat'] = df_pivot['Vaatajad, tuhat'].apply(lambda x: str(x).replace('.', ',') if isinstance(x, str) else x)
df_pivot['Teatriskäigud 1000 elaniku kohta'] = df_pivot['Teatriskäigud 1000 elaniku kohta'].apply(lambda x: str(x).replace('.', ',') if isinstance(x, str) else x)

# Step 10: Adjust Year display
df_pivot.sort_values(by=['Year', 'Category'], inplace=True)
df_pivot.reset_index(drop=True, inplace=True)
df_pivot['Year'] = df_pivot['Year'].mask(df_pivot['Year'].duplicated())
df_pivot['Year'].fillna('', inplace=True)

# Step 11: Display the final DataFrame
print(df_pivot.to_string(index=False))

output_path = '/home/aleksis/PycharmProjects/DatascienceGroupProject/TheaterDataAnalysis/data/processed'
df_pivot.to_csv(output_path, index=False, encoding='utf-8')


/tmp/ipykernel_111679/3903985261.py:42: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final['value'] = df_final['value'].replace({None: np.nan})
/tmp/ipykernel_111679/3903985261.py:70: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '..' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_pivot.fillna('..', inplace=True)
/tmp/ipykernel_111679/3903985261.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beha

   Year     Category Lavastused ..uuslavastused Etendused ..külalisetendused Vaatajad, tuhat
 2007.0        Kokku      415.0           165.0   4 752.0            1 287.0         1 010,9
              Lapsed       92.0            31.0   1 233.0              430.0           238,0
              Noored       46.0            17.0     440.0              137.0            90,9
        Täiskasvanud      277.0           117.0   3 079.0              720.0           681,9
 2008.0        Kokku      400.0           157.0   4 635.0            1 304.0           983,1
              Lapsed       82.0            29.0   1 154.0              404.0           229,3
              Noored       43.0            20.0     436.0              119.0            88,1
        Täiskasvanud      275.0           108.0   3 045.0              781.0           665,8
 2009.0        Kokku      401.0           153.0   4 731.0            1 125.0           873,8
              Lapsed       92.0            32.0   1 327.0             

KeyError: "['Kultuuriüritused'] not in index"